In [1]:
# jupyter中使用hyperparams需要添加
import sys
sys.argv = ['run.py']

根据经纬度映射出区域号.
- xlim:经度范围
- ylim:纬度范围

In [2]:
def get_regionID(longitude, latitude):
    xlim = [111.55, 113.37]
    ylim = [35.12, 36.3]
    grid_length = 0.001
    grid_width = 0.001
    regionID = (xlim[1] - xlim[0]) // grid_length * (latitude - ylim[0]) // grid_width + (
            longitude - xlim[0]) // grid_length + 1
    return int(regionID)

get_regionID(111.653, 35.232)

203944

轨迹过滤：过滤掉timeslots小于m的轨迹，过滤掉总条数少于5条的轨迹。

In [3]:
import os
file_list = sorted(os.listdir('./data/Geolife/Data'))
file_num = len(file_list)
file_list = file_list[1:]


for user_id in file_list:
    u_path = os.getcwd() + "/data" + "/Geolife" + "/Data/" + user_id + "/Trajectory"



# path = os.getcwd() + "/data" + "/Geolife" + "/Data" + "/001" + "/Trajectory"
# print(path)
# user_path = os.scandir(path)
# print(user_path)

In [4]:
output_path = './Results/size%d' % (1)
if not os.path.exists(output_path):
    # 目录不存在，进行创建操作
    os.makedirs(output_path)

  

In [1]:
import pandas as pd
import datatime

time0 = datetime.now()
start_time = datetime.now()
result_tmp = []
data_tmp = pd.DataFrame()
data_all = pd.DataFrame()  
# data_all.columns=['UID','timestamp','longitude','latitude']
for filename in file_list:
    user_path = "./data" + "/Geolife" + "/Data/" + user_id + "/Trajectory"
    f = open(user_path, 'r+')
    for line in f.readlines()[6::10]:  # readlines以列表输出文件内容
        line = line.strip().split(",")
        print(line)
    f.close()
    print(filename + 'is read.')
end_time = datetime.now()

ModuleNotFoundError: No module named 'datatime'